# Weather Dataset Preprocessing and feature Engineering

In [1]:
%AddJar file:///home/jovyan/work/apps/Emiasd-Flight-Data-Analysis.jar

Starting download from file:///home/jovyan/work/apps/Emiasd-Flight-Data-Analysis.jar
Finished download of Emiasd-Flight-Data-Analysis.jar
Using cached version of Emiasd-Flight-Data-Analysis.jar


In [2]:
import org.apache.spark.sql.SparkSession
import com.flightdelay.config.{AppConfiguration, ConfigurationLoader}
import com.flightdelay.data.loaders.FlightDataLoader

//Env Configuration
val args: Array[String] = Array("jupyter")

val spark = SparkSession.builder()
  .config(sc.getConf)
  .getOrCreate()

// Rendre la session Spark implicite
implicit val session = spark
implicit val configuration: AppConfiguration = ConfigurationLoader.loadConfiguration(args)

val processedWeatherPath = s"${configuration.common.output.basePath}/common/data/processed_weather.parquet"
val weatherDF = spark.read.parquet(processedWeatherPath)

args = Array(jupyter)
spark = org.apache.spark.sql.SparkSession@7ff83735
session = org.apache.spark.sql.SparkSession@7ff83735
configuration = AppConfiguration(local,CommonConfig(42,true,debug,false,false,DataConfig(/home/jovyan/work/data,FileConfig(/home/jovyan/work/data/FLIGHT-3Y/Flights/201201*.csv),FileConfig(/home/jovyan/work/data/FLIGHT-3Y/Weather/20101*.txt),FileConfig(/home/jovyan/work/data/FLIGHT-3Y/wban_airport_timezone.csv)),OutputConfig(/home/jovyan/work/output,FileConfig(/home/jovyan/work/o...


AppConfiguration(local,CommonConfig(42,true,debug,false,false,DataConfig(/home/jovyan/work/data,FileConfig(/home/jovyan/work/data/FLIGHT-3Y/Flights/201201*.csv),FileConfig(/home/jovyan/work/data/FLIGHT-3Y/Weather/20101*.txt),FileConfig(/home/jovyan/work/data/FLIGHT-3Y/wban_airport_timezone.csv)),OutputConfig(/home/jovyan/work/output,FileConfig(/home/jovyan/work/o...

In [3]:
weatherDF.count()

1549320

In [4]:
weatherDF.printSchema

root
 |-- feature_pressure_bucket: string (nullable = true)
 |-- weather_hazard_level: integer (nullable = true)
 |-- feature_weather_severity_index: double (nullable = true)
 |-- RelativeHumidity: double (nullable = true)
 |-- feature_visibility_category: string (nullable = true)
 |-- feature_flight_category_ordinal: integer (nullable = true)
 |-- feature_visibility_risk_score: double (nullable = true)
 |-- feature_requires_cat_ii: integer (nullable = true)
 |-- extracted_codes: string (nullable = true)
 |-- feature_operations_risk_level: integer (nullable = true)
 |-- feature_weather_type: string (nullable = true)
 |-- feature_pressure_vis_combo_bin: string (nullable = true)
 |-- feature_icing_risk_level: integer (nullable = true)
 |-- feature_weather_intensity: string (nullable = true)
 |-- feature_most_critical_sky: string (nullable = true)
 |-- press_change_abs: double (nullable = true)
 |-- feature_temp_dewpoint_spread: double (nullable = true)
 |-- feature_is_very_low_visibility

## Station Type

In [5]:
weatherDF
  .select("StationType")
  .distinct()
  .show(false)

org.apache.spark.sql.catalyst.ExtendedAnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column or function parameter with name `StationType` cannot be resolved. Did you mean one of the following? [`Altimeter`, `Date`, `StationPressure`, `has_snow`, `Time`].;
'Project ['StationType]
+- Relation [WindDirection#0,PressureChange#1,feature_pressure_bucket#2,has_hail#3,DewPointFarenheit#4,has_visible_moisture#5,WetBulbCelsius#6,feature_flight_category#7,DryBulbFarenheit#8,weather_hazard_level#9,feature_weather_severity_index#10,has_obscuration#11,feature_visibility_miles#12,has_precipitation#13,RelativeHumidity#14,feature_visibility_category#15,weather_intensity#16,feature_has_broken#17,has_thunderstorm#18,feature_is_ifr_conditions#19,feature_flight_category_ordinal#20,Altimeter#21,DryBulbCelsius#22,feature_visibility_risk_score#23,... 41 more fields] parquet


## SkyConditionFeatures

In [6]:
var addedFeatureColumns = Seq(
"feature_most_critical_sky",
"feature_num_cloud_layers",
"feature_cloud_risk_score",
"feature_has_overcast",
"feature_has_broken",
"feature_has_obscured",
"feature_is_clear",
"feature_lowest_cloud_height",
"feature_ceiling",
"feature_has_low_ceiling"
)

var df = weatherDF
  .select(addedFeatureColumns.head, addedFeatureColumns.tail: _*)
df.show(1, 1000, true)

-RECORD 0----------------------------
 feature_most_critical_sky   | FEW   
 feature_num_cloud_layers    | 1     
 feature_cloud_risk_score    | 1.0   
 feature_has_overcast        | 0     
 feature_has_broken          | 0     
 feature_has_obscured        | 0     
 feature_is_clear            | 0     
 feature_lowest_cloud_height | 9000  
 feature_ceiling             | 99999 
 feature_has_low_ceiling     | 0     
only showing top 1 row



lastException = null
addedFeatureColumns = List(feature_most_critical_sky, feature_num_cloud_layers, feature_cloud_risk_score, feature_has_overcast, feature_has_broken, feature_has_obscured, feature_is_clear, feature_lowest_cloud_height, feature_ceiling, feature_has_low_ceiling)
df = [feature_most_critical_sky: string, feature_num_cloud_layers: int ... 8 more fields]


[feature_most_critical_sky: string, feature_num_cloud_layers: int ... 8 more fields]

## VisibilityFeatures

In [7]:
addedFeatureColumns = Seq(
"feature_visibility_miles",
"feature_visibility_km",
"feature_visibility_category",
"feature_visibility_risk_score",
"feature_is_low_visibility",
"feature_is_very_low_visibility",
"feature_visibility_normalized",
"feature_visibility_inverse"
)

val df = weatherDF
  .select(addedFeatureColumns.head, addedFeatureColumns.tail: _*)
df.show(1, 1000, true)

addedFeatureColumns = List(feature_visibility_miles, feature_visibility_km, feature_visibility_category, feature_visibility_risk_score, feature_is_low_visibility, feature_is_very_low_visibility, feature_visibility_normalized, feature_visibility_inverse)
df = [feature_visibility_miles: double, feature_visibility_km: double ... 6 more fields]


-RECORD 0-------------------------------
 feature_visibility_miles       | 1.0   
 feature_visibility_km          | 1.609 
 feature_visibility_category    | IFR   
 feature_visibility_risk_score  | 3.0   
 feature_is_low_visibility      | 1     
 feature_is_very_low_visibility | 0     
 feature_visibility_normalized  | 0.1   
 feature_visibility_inverse     | 1.0   
only showing top 1 row



[feature_visibility_miles: double, feature_visibility_km: double ... 6 more fields]

In [8]:
// Obtenir les valeurs distinctes
var distinctValues = df.select("feature_visibility_category")
  .distinct()
  .orderBy("feature_visibility_category")

// Afficher les résultats
distinctValues.show()

+---------------------------+
|feature_visibility_category|
+---------------------------+
|                        IFR|
|                    IFR_LOW|
|                       LIFR|
|                       MVFR|
|                        VFR|
|                   VFR_HIGH|
+---------------------------+



distinctValues = [feature_visibility_category: string]


[feature_visibility_category: string]

## WeatherInteractionFeatures

In [9]:
addedFeatureColumns = Seq(
"feature_weather_severity_index",
"feature_is_vfr_conditions",
"feature_is_ifr_conditions",
"feature_requires_cat_ii",
"feature_operations_risk_level"
)

val df = weatherDF
  .select(addedFeatureColumns.head, addedFeatureColumns.tail: _*)
df.show(1, 1000, true)

-RECORD 0--------------------------------------------
 feature_weather_severity_index | 2.1999999999999997 
 feature_is_vfr_conditions      | 0                  
 feature_is_ifr_conditions      | 1                  
 feature_requires_cat_ii        | 0                  
 feature_operations_risk_level  | 2                  
only showing top 1 row



addedFeatureColumns = List(feature_weather_severity_index, feature_is_vfr_conditions, feature_is_ifr_conditions, feature_requires_cat_ii, feature_operations_risk_level)
df = [feature_weather_severity_index: double, feature_is_vfr_conditions: int ... 3 more fields]


[feature_weather_severity_index: double, feature_is_vfr_conditions: int ... 3 more fields]

## WeatherTypeFeatureGenerator

In [10]:
addedFeatureColumns = Seq(
"intensity_heavy",
"intensity_light",
"weather_intensity",
"has_thunderstorm",
"has_freezing_precip",
"has_freezing",
"has_precipitation",
"has_obscuration",
"has_visible_moisture",
"has_hazardous",
"has_rain",
"has_snow",
"has_hail",
"extracted_codes",
"weather_hazard_level",
"Icing_Risk_Flag",
"Icing_Risk_Level"
)

val df = weatherDF
  .select(addedFeatureColumns.head, addedFeatureColumns.tail: _*)
df.show(1, 1000, true)

addedFeatureColumns = List(intensity_heavy, intensity_light, weather_intensity, has_thunderstorm, has_freezing_precip, has_freezing, has_precipitation, has_obscuration, has_visible_moisture, has_hazardous, has_rain, has_snow, has_hail, extracted_codes, weather_hazard_level, Icing_Risk_Flag, Icing_Risk_Level)
df = [intensity_heavy: int, intensity_light: int ... 15 more fields]


-RECORD 0------------------------
 intensity_heavy      | 0        
 intensity_light      | 0        
 weather_intensity    | moderate 
 has_thunderstorm     | 0        
 has_freezing_precip  | 0        
 has_freezing         | 0        
 has_precipitation    | 0        
 has_obscuration      | 0        
 has_visible_moisture | 0        
 has_hazardous        | 0        
 has_rain             | 0        
 has_snow             | 0        
 has_hail             | 0        
 extracted_codes      |          
 weather_hazard_level | 0        
 Icing_Risk_Flag      | 1        
 Icing_Risk_Level     | 1        
only showing top 1 row



[intensity_heavy: int, intensity_light: int ... 15 more fields]

In [11]:
// Obtenir les valeurs distinctes
val distinctValues = df.select("weather_intensity")
  .distinct()
  .orderBy("weather_intensity")

// Afficher les résultats
distinctValues.show()

+-----------------+
|weather_intensity|
+-----------------+
|            heavy|
|            light|
|         moderate|
+-----------------+



distinctValues = [weather_intensity: string]


[weather_intensity: string]

In [12]:
import org.apache.spark.sql.functions.col

// Filtrer sur weather_intensity == "heavy"
val heavyWeatherDF = df.filter(col("weather_intensity") === "heavy")

// Afficher le résultat
heavyWeatherDF.show(1, 10000, true)

-RECORD 0---------------------
 intensity_heavy      | 1     
 intensity_light      | 0     
 weather_intensity    | heavy 
 has_thunderstorm     | 1     
 has_freezing_precip  | 0     
 has_freezing         | 0     
 has_precipitation    | 1     
 has_obscuration      | 0     
 has_visible_moisture | 1     
 has_hazardous        | 1     
 has_rain             | 1     
 has_snow             | 0     
 has_hail             | 0     
 extracted_codes      | TSRA  
 weather_hazard_level | 3     
 Icing_Risk_Flag      | 0     
 Icing_Risk_Level     | 0     
only showing top 1 row



heavyWeatherDF = [intensity_heavy: int, intensity_light: int ... 15 more fields]


[intensity_heavy: int, intensity_light: int ... 15 more fields]

## Original DF

In [14]:
var featureColumns = Seq(
"DryBulbCelsius",
//"DryBulbCelsiusFlag",
"DryBulbFarenheit",
//"DryBulbFarenheitFlag"
)

val df = weatherDF
  .select(featureColumns.head, featureColumns.tail: _*)
df.show(1, 1000, true)

lastException = null
featureColumns = List(DryBulbCelsius, DryBulbFarenheit)
df = [DryBulbCelsius: double, DryBulbFarenheit: double]


-RECORD 0----------------
 DryBulbCelsius   | -7.2 
 DryBulbFarenheit | 19.0 
only showing top 1 row



[DryBulbCelsius: double, DryBulbFarenheit: double]

In [30]:
featureColumns = Seq(
"DewPointCelsius",
"DewPointCelsiusFlag",
"DewPointFarenheit",
"DewPointFarenheitFlag"
)

val df = weatherDF
  .select(featureColumns.head, featureColumns.tail: _*)
df.show(1, 1000, true)

featureColumns = List(DewPointCelsius, DewPointCelsiusFlag, DewPointFarenheit, DewPointFarenheitFlag)
df = [DewPointCelsius: double, DewPointCelsiusFlag: string ... 2 more fields]


-RECORD 0----------------------
 DewPointCelsius       | -12.0 
 DewPointCelsiusFlag   |       
 DewPointFarenheit     | 10.0  
 DewPointFarenheitFlag |       
only showing top 1 row



[DewPointCelsius: double, DewPointCelsiusFlag: string ... 2 more fields]

In [20]:
import org.apache.spark.sql.functions._
weatherDF.select(
  sum(col("DewPointCelsius").isNull.cast("int")).as("null_Celsius"),
  sum(col("DewPointFarenheit").isNull.cast("int")).as("null_Farenheit")
).show()

+------------+--------------+
|null_Celsius|null_Farenheit|
+------------+--------------+
|        8937|          8937|
+------------+--------------+



In [25]:
import org.apache.spark.sql.expressions.Window
import org.apache.spark.sql.functions._

val wForward = Window
  .partitionBy("WBAN")
  .orderBy("Date", "Time")
  .rowsBetween(Window.unboundedPreceding, Window.currentRow)

val dfForward = weatherDF
  .withColumn("DewPointCelsius_ff",
    last("DewPointCelsius", ignoreNulls = true).over(wForward))
  .withColumn("DewPointFarenheit_ff",
    last("DewPointFarenheit", ignoreNulls = true).over(wForward))

val wBackward = Window
  .partitionBy("WBAN")
  .orderBy(col("Date").desc, col("Time").desc)
  .rowsBetween(Window.unboundedPreceding, Window.currentRow)

val dfBoth = dfForward
  .withColumn("DewPointCelsius_bf",
    last("DewPointCelsius", ignoreNulls = true).over(wBackward))
  .withColumn("DewPointFarenheit_bf",
    last("DewPointFarenheit", ignoreNulls = true).over(wBackward))

val weatherFilled2 = dfBoth
  .withColumn(
    "DewPointCelsius_filled",
    coalesce(col("DewPointCelsius_ff"), col("DewPointCelsius_bf"))
  )
  .withColumn(
    "DewPointFarenheit_filled",
    coalesce(col("DewPointFarenheit_ff"), col("DewPointFarenheit_bf"))
  )

weatherFilled2.select(
  sum(col("DewPointCelsius_filled").isNull.cast("int")).as("null_Celsius"),
  sum(col("DewPointFarenheit_filled").isNull.cast("int")).as("null_Farenheit")
).show()

+------------+--------------+
|null_Celsius|null_Farenheit|
+------------+--------------+
|           0|             0|
+------------+--------------+



wForward = org.apache.spark.sql.expressions.WindowSpec@590316b3
dfForward = [WindDirection: double, PressureChange: double ... 65 more fields]
wBackward = org.apache.spark.sql.expressions.WindowSpec@2ce23f11
dfBoth = [WindDirection: double, PressureChange: double ... 67 more fields]
weatherFilled2 = [WindDirection: double, PressureChange: double ... 69 more fields]


[WindDirection: double, PressureChange: double ... 69 more fields]

In [26]:
featureColumns = Seq(
"WetBulbCelsius",
//"WetBulbCelsiusFlag",
"WetBulbFarenheit",
//"WetBulbFarenheitFlag"
)

val df = weatherDF
  .select(featureColumns.head, featureColumns.tail: _*)
df.show(1, 1000, true)

-RECORD 0----------------
 WetBulbCelsius   | -7.9 
 WetBulbFarenheit | 18.0 
only showing top 1 row



featureColumns = List(WetBulbCelsius, WetBulbFarenheit)
df = [WetBulbCelsius: double, WetBulbFarenheit: double]


[WetBulbCelsius: double, WetBulbFarenheit: double]

In [27]:
featureColumns = Seq(
"RelativeHumidity",
//"RelativeHumidityFlag"
)

val df = weatherDF
  .select(featureColumns.head, featureColumns.tail: _*)
df.show(1, 1000, true)

-RECORD 0----------------
 RelativeHumidity | 84.0 
only showing top 1 row



featureColumns = List(RelativeHumidity)
df = [RelativeHumidity: double]


[RelativeHumidity: double]

In [29]:
featureColumns = Seq(
"HourlyPrecip",
//"HourlyPrecipFlag"
)

val df = weatherDF
  .select(featureColumns.head, featureColumns.tail: _*)
df.show(1, 1000, true)

-RECORD 0------------
 HourlyPrecip | NULL 
only showing top 1 row



lastException = null
featureColumns = List(HourlyPrecip)
df = [HourlyPrecip: double]


[HourlyPrecip: double]

In [30]:
featureColumns = Seq(
"WindSpeed",
//"WindSpeedFlag",
"WindDirection",
//"WindDirectionFlag",
"ValueForWindCharacter",
//"ValueForWindCharacterFlag"
)

val df = weatherDF
  .select(featureColumns.head, featureColumns.tail: _*)
df.show(1, 1000, true)

-RECORD 0----------------------
 WindSpeed             | 5.0   
 WindDirection         | 220.0 
 ValueForWindCharacter | NULL  
only showing top 1 row



featureColumns = List(WindSpeed, WindDirection, ValueForWindCharacter)
df = [WindSpeed: double, WindDirection: double ... 1 more field]


[WindSpeed: double, WindDirection: double ... 1 more field]

In [37]:
val featureColumns = Seq(
    //"SkyCondition",
    //"Visibility",
    //"WeatherType",
    "extracted_codes"
)

val df = weatherDF
  .select(featureColumns.head, featureColumns.tail: _*)
  .where(col("extracted_codes").isNotNull)

df.show(100, 1000, false)

+---------------+
|extracted_codes|
+---------------+
|               |
|               |
|               |
|               |
|               |
|               |
|               |
|               |
|               |
|               |
|               |
|               |
|               |
|               |
|               |
|               |
|               |
|               |
|             HZ|
|               |
|               |
|               |
|               |
|               |
|               |
|               |
|               |
|               |
|               |
|               |
|               |
|               |
|               |
|               |
|               |
|               |
|               |
|          RA BR|
|               |
|               |
|             HZ|
|               |
|               |
|               |
|             HZ|
|               |
|               |
|               |
|               |
|               |
|               |
|               |
|         

featureColumns = List(extracted_codes)
df = [extracted_codes: string]


[extracted_codes: string]

In [11]:
val featureColumns = Seq(
"WindSpeed",
"WindSpeedFlag",
"WindDirection",
"WindDirectionFlag",
"ValueForWindCharacter",
"ValueForWindCharacterFlag"
)

val df = weatherDF
  .select(featureColumns.head, featureColumns.tail: _*)
df.show(100)

featureColumns = List(WindSpeed, WindSpeedFlag, WindDirection, WindDirectionFlag, ValueForWindCharacter, ValueForWindCharacterFlag)
df = [WindSpeed: double, WindSpeedFlag: string ... 4 more fields]


+---------+-------------+-------------+-----------------+---------------------+-------------------------+
|WindSpeed|WindSpeedFlag|WindDirection|WindDirectionFlag|ValueForWindCharacter|ValueForWindCharacterFlag|
+---------+-------------+-------------+-----------------+---------------------+-------------------------+
|     21.0|             |        360.0|                 |                   30|                         |
|      5.0|             |        200.0|                 |                     |                         |
|     NULL|             |         NULL|                 |                     |                         |
|      3.0|             |        260.0|                 |                     |                         |
|      3.0|             |        230.0|                 |                     |                         |
|      2.0|             |         NULL|                 |                     |                         |
|      0.0|             |          0.0|       

[WindSpeed: double, WindSpeedFlag: string ... 4 more fields]

In [14]:
val featureColumns = Seq(
"RelativeHumidity",
"StationPressure",
"PressureTendency",
"PressureChange",
"SeaLevelPressure",
"RecordType",
"HourlyPrecip",
"Altimeter"
)

val df = weatherDF
  .select(featureColumns.head, featureColumns.tail: _*)
df.show(100)

featureColumns = List(RelativeHumidity, StationPressure, PressureTendency, PressureChange, SeaLevelPressure, RecordType, HourlyPrecip, Altimeter)
df = [RelativeHumidity: double, StationPressure: double ... 6 more fields]


+----------------+---------------+----------------+--------------+----------------+----------+------------+---------+
|RelativeHumidity|StationPressure|PressureTendency|PressureChange|SeaLevelPressure|RecordType|HourlyPrecip|Altimeter|
+----------------+---------------+----------------+--------------+----------------+----------+------------+---------+
|            77.0|          25.86|                |          NULL|           30.25|        AA|            |    30.18|
|            63.0|          26.91|                |          NULL|           29.93|        AA|            |    30.03|
|            NULL|           NULL|                |          NULL|               M|     CRN05|            |     NULL|
|            80.0|          25.58|                |          NULL|           29.90|        AA|           T|    29.97|
|            93.0|          29.32|                |          NULL|               M|        SP|            |    29.88|
|            NULL|           NULL|                |     

[RelativeHumidity: double, StationPressure: double ... 6 more fields]

In [16]:
import org.apache.spark.sql.{DataFrame, Row, SparkSession}
import org.apache.spark.sql.functions._
import org.apache.spark.sql.types._

def summarizeMAndNumeric(df: DataFrame)(implicit spark: SparkSession): DataFrame = {
  val numericRegex = "^[+-]?(\\d+\\.?\\d*|\\.\\d+)([eE][+-]?\\d+)?$"

  // 1) Agrégations en un seul scan
  val aggExprs = df.columns.flatMap { c =>
    val s = col(c).cast("string")
    Seq(
      sum( when(s === "M", 1).otherwise(0) ).alias(s"${c}__m_count"),
      // valeurs non nulles, ≠ "M", non-numériques
      sum( when(s.isNotNull && s =!= "M" && !s.rlike(numericRegex), 1).otherwise(0) )
        .alias(s"${c}__non_numeric_other_count"),
      sum( when(s.isNotNull, 1).otherwise(0) ).alias(s"${c}__nonnull_count")
    )
  }

  val aggRow = df.agg(aggExprs.head, aggExprs.tail: _*).first()

  // 2) Construire un petit DF récapitulatif côté driver (très petit)
  val rows: Seq[Row] = df.columns.map { c =>
    val mCount   = aggRow.getAs[Number](s"${c}__m_count").longValue()
    val badCount = aggRow.getAs[Number](s"${c}__non_numeric_other_count").longValue()
    val nonNull  = aggRow.getAs[Number](s"${c}__nonnull_count").longValue()
    val ok       = (mCount > 0) && (badCount == 0L)
    Row(c, mCount, badCount, nonNull, ok)
  }

  val schema = StructType(Seq(
    StructField("column", StringType, false),
    StructField("m_count", LongType, false),
    StructField("non_numeric_other_count", LongType, false),
    StructField("nonnull_count", LongType, false),
    StructField("ok_pattern", BooleanType, false) // true => contient "M" et le reste est NULL/numérique
  ))

  spark.createDataFrame(spark.sparkContext.parallelize(rows), schema)
}

val summaryDF = summarizeMAndNumeric(weatherDF)
summaryDF.orderBy(desc("ok_pattern"), desc("m_count")).show(truncate = false)

+---------------------------+-------+-----------------------+-------------+----------+
|column                     |m_count|non_numeric_other_count|nonnull_count|ok_pattern|
+---------------------------+-------+-----------------------+-------------+----------+
|SeaLevelPressure           |5888816|0                      |11236398     |true      |
|SkyCondition               |1481170|9755228                |11236398     |false     |
|RecordType                 |0      |11236398               |11236398     |false     |
|feature_most_critical_sky  |0      |11236398               |11236398     |false     |
|RecordTypeFlag             |0      |11236398               |11236398     |false     |
|feature_is_clear           |0      |11236398               |11236398     |false     |
|HourlyPrecip               |0      |10907635               |11236398     |false     |
|feature_num_cloud_layers   |0      |0                      |11236398     |false     |
|HourlyPrecipFlag           |0      |112363

summaryDF = [column: string, m_count: bigint ... 3 more fields]


summarizeMAndNumeric: (df: org.apache.spark.sql.DataFrame)(implicit spark: org.apache.spark.sql.SparkSession)org.apache.spark.sql.DataFrame


[column: string, m_count: bigint ... 3 more fields]